# Churn Prediction with XGBoost on Marketing Data

### Inportando os pacotes necessários para análise.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
sns.set_style("darkgrid")

import warnings
warnings.filterwarnings("ignore")

PATH = '/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive'

import os
for dirname, _, filenames in os.walk(PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/product.csv
/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/transaction_data.csv
/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/causal_data.csv
/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/campaign_table.csv
/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/coupon.csv
/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/hh_demographic.csv
/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/campaign_desc.csv
/home/leandro/Documentos/archive_Churn_Predicition_with_XGBoost_on_Marketing_Data/archive/coupon_redempt.csv


### Análise exploratória de dados
#### O conjunto de dados consiste em oito tabelas em arquivos .csv separados, examinaremos seis delas que usaremos neste bloco de notas. 
#### O conjunto de dados cobre transações de compra de um período de dois anos de 2.500 famílias.
#### Também estão disponíveis informações demográficas de famílias, informações de resgate de campanha e cupons. 
#### Na fase de modelagem, juntaremos essas tabelas para fazer nosso conjunto de dados final.
- Descrições de campanha (campaigndesc.csv)
- Campanhas (campaign_table.csv)
- Cupons (coupon.csv)
- Resgates de cupons (coupon_redempt.csv)
- Transações (transaction_data.csv)
- Dados demográficos (hh_demographic.csv)

### Descrições de campanha
#### Os dados de descrição da campanha são uma tabela de consulta, contendo os dias de início e término de cada campanha (30 campanhas no total). Também fornece qual campanha pertence a qual categoria (Tipo A, B e C).

In [2]:
# Criando um dicionário para armazenar as tabelas

table_dictionary = {}

for dirname, _, filenames in os.walk(PATH):
    for filename in filenames:
        df_path = os.path.join(dirname, filename)
        name = filename.split('.csv')[0]
        table_dictionary[name] = pd.read_csv(df_path)

In [3]:
lista_de_tabelas = list(table_dictionary.keys())
lista_de_tabelas

['product',
 'transaction_data',
 'causal_data',
 'campaign_table',
 'coupon',
 'hh_demographic',
 'campaign_desc',
 'coupon_redempt']

In [4]:
# Verificando se exitem valores Na nas tabelas

for table in lista_de_tabelas:
    df = table_dictionary[table]
    df_na = df.isna().sum()
    val_na = df_na.values.any()
    if val_na != 0:
        print(df_na)
    else:
        print(f'A tabela {table} não tem valores Na')

A tabela product não tem valores Na
A tabela transaction_data não tem valores Na
A tabela causal_data não tem valores Na
A tabela campaign_table não tem valores Na
A tabela coupon não tem valores Na
A tabela hh_demographic não tem valores Na
A tabela campaign_desc não tem valores Na
A tabela coupon_redempt não tem valores Na


In [5]:
# Visualizando o shape de cada tabela e os valores únicos de cada coluna das tabelas
for table in lista_de_tabelas:
    df = table_dictionary[table]
    df_nunique = df.nunique()
    print(f'shape -- {df.shape}')
    print('-----------')
    print(f'|Tabela {table}|')
    print('-----------')
    print('| colunas |')
    print('-----------')
    print(df_nunique,'\n')

shape -- (92353, 7)
-----------
|Tabela product|
-----------
| colunas |
-----------
PRODUCT_ID              92353
MANUFACTURER             6476
DEPARTMENT                 44
BRAND                       2
COMMODITY_DESC            308
SUB_COMMODITY_DESC       2383
CURR_SIZE_OF_PRODUCT     4345
dtype: int64 

shape -- (2595732, 12)
-----------
|Tabela transaction_data|
-----------
| colunas |
-----------
household_key          2500
BASKET_ID            276484
DAY                     711
PRODUCT_ID            92339
QUANTITY              11746
SALES_VALUE            5502
STORE_ID                582
RETAIL_DISC            2414
TRANS_TIME             1440
WEEK_NO                 102
COUPON_DISC             415
COUPON_MATCH_DISC        80
dtype: int64 

shape -- (36786524, 5)
-----------
|Tabela causal_data|
-----------
| colunas |
-----------
PRODUCT_ID    68377
STORE_ID        115
WEEK_NO          93
display          10
mailer           11
dtype: int64 

shape -- (7208, 3)
-----------
|Tab

In [6]:
# função para identificar quantidade de linhas vazias por coluna do tipo object, gerando tabela
def func_none_values(df):
    df_t = pd.DataFrame(columns=['coluna','n_linhas_nulas', 'percent_linhas_nulas'])
    list_cols_object = df.select_dtypes(include=['object']).columns
    for col in list_cols_object:
        df_null = df[~df[col].str.contains('[\w]')]
        rows_none = df_null.shape[0]
        rows = df.shape[0]
        percent_rows_none = '{:.3f}'.format(rows_none/rows)
        df_t = df_t.append({'coluna': col ,
                            'n_linhas_nulas':rows_none, 
                            'percent_linhas_nulas':percent_rows_none},
                           ignore_index=True)
        if df_t.n_linhas_nulas.sum() == 0:
            return None
    return df_t   

In [7]:
# Verificando se as tabelas tem valores vazios.
# Foi constatado que somente a tabela Product apresentou linhas vazias e somente coluna CURR_SIZE_OF_PRODUCT
# apresentou um valor relevante de 33% de valores vazios 
for table in lista_de_tabelas:
    df = table_dictionary[table]
    try:
        if len(func_none_values(df)) > 0:
            print(f'\n{table}')
            print(func_none_values(df))
    except:
        pass


product
                 coluna n_linhas_nulas percent_linhas_nulas
0            DEPARTMENT             15                0.000
1                 BRAND              0                0.000
2        COMMODITY_DESC             15                0.000
3    SUB_COMMODITY_DESC             15                0.000
4  CURR_SIZE_OF_PRODUCT          30607                0.331


In [8]:
table_dictionary.keys()

dict_keys(['product', 'transaction_data', 'causal_data', 'campaign_table', 'coupon', 'hh_demographic', 'campaign_desc', 'coupon_redempt'])

In [9]:
table_dictionary['campaign_desc']

,DESCRIPTION,CAMPAIGN,START_DAY,END_DAY
0,TypeB,24,659,719
1,TypeC,15,547,708
2,TypeB,25,659,691
3,TypeC,20,615,685
4,TypeB,23,646,684
5,TypeB,21,624,656
6,TypeB,22,624,656
7,TypeA,18,587,642
8,TypeB,19,603,635
9,TypeB,17,575,607


In [10]:
df = table_dictionary['transaction_data']

df.nunique()

household_key          2500
BASKET_ID            276484
DAY                     711
PRODUCT_ID            92339
QUANTITY              11746
SALES_VALUE            5502
STORE_ID                582
RETAIL_DISC            2414
TRANS_TIME             1440
WEEK_NO                 102
COUPON_DISC             415
COUPON_MATCH_DISC        80
dtype: int64

In [11]:
df[df.QUANTITY == 2]

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
16,1130,26984905972,1,833715,2,0.34,31642,-0.32,1340,1,0.0,0.0
17,1130,26984905972,1,866950,2,0.34,31642,-0.32,1340,1,0.0,0.0
18,1130,26984905972,1,1022843,2,0.34,31642,-0.32,1340,1,0.0,0.0
20,1130,26984905972,1,1071333,2,0.34,31642,-0.32,1340,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2595651,2125,42291245957,711,1977026,2,1.88,795,0.00,1427,102,0.0,0.0
2595666,2125,42291245957,711,6607550,2,5.18,795,0.00,1427,102,0.0,0.0
2595685,1037,42291971719,711,72816,2,23.98,3182,-1.00,1540,102,0.0,0.0
2595704,1369,42302712189,711,959737,2,25.98,446,0.00,1730,102,0.0,0.0


In [12]:
df[df.QUANTITY == 1].shape[0]/df.shape[0]

0.7901304911292846

In [13]:
df.QUANTITY.value_counts(normalize=True)

1        7.901305e-01
2        1.434559e-01
3        2.579696e-02
4        1.489638e-02
0        5.572994e-03
             ...     
4117     3.852478e-07
2990     3.852478e-07
16943    3.852478e-07
5932     3.852478e-07
3989     3.852478e-07
Name: QUANTITY, Length: 11746, dtype: float64

In [14]:
df = table_dictionary['product']
col = 'CURR_SIZE_OF_PRODUCT'

df_null_rows = df[~df[col].str.contains('[\w]')]

In [15]:
df_null_rows

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
8,26636,69,PASTRY,Private,BREAKFAST SWEETS,SW GDS: SW ROLLS/DAN,
11,26889,32,DRUG GM,National,MAGAZINE,TV/MOVIE-MAGAZINE,
13,27021,2,GROCERY,National,AIR CARE,AIR CARE - AEROSOLS,
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


In [16]:
df.nunique()

PRODUCT_ID              92353
MANUFACTURER             6476
DEPARTMENT                 44
BRAND                       2
COMMODITY_DESC            308
SUB_COMMODITY_DESC       2383
CURR_SIZE_OF_PRODUCT     4345
dtype: int64

In [17]:
df_null_rows.nunique()

PRODUCT_ID              30607
MANUFACTURER             4367
DEPARTMENT                 40
BRAND                       2
COMMODITY_DESC            279
SUB_COMMODITY_DESC       1597
CURR_SIZE_OF_PRODUCT        1
dtype: int64

In [18]:
df.DEPARTMENT.value_counts()

GROCERY            39021
DRUG GM            31529
PRODUCE             3118
COSMETICS           3011
NUTRITION           2914
MEAT                2544
MEAT-PCKGD          2427
DELI                2354
PASTRY              2149
FLORAL               938
SEAFOOD-PCKGD        563
MISC. TRANS.         490
SPIRITS              377
SEAFOOD              369
GARDEN CENTER        128
RESTAURANT           102
MISC SALES TRAN       88
SALAD BAR             48
COUP/STR & MFG        39
TRAVEL & LEISUR       28
FROZEN GROCERY        23
KIOSK-GAS             16
                      15
CHEF SHOPPE           14
RX                     9
CNTRL/STORE SUP        4
DAIRY DELI             3
TOYS                   3
POSTAL CENTER          3
GM MERCH EXP           3
VIDEO RENTAL           3
AUTOMOTIVE             2
PHOTO                  2
VIDEO                  2
PROD-WHS SALES         2
DELI/SNACK BAR         2
CHARITABLE CONT        2
GRO BAKERY             2
PORK                   1
HBC                    1
